# PEC Monte Carlo simulation

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

In the next few steps we import all relevant data and set up the melt initial FeO prediction model. These steps are identical to the [PEC model example](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html#)

Import melt inclusion and olivine data:

In [2]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
olivine = mp.read_olivine(olivine_file, index_col=["name"])

Import inclusion internal pressures:

In [3]:
pressure_file ="./data/pressure.csv"
pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

Set up the melt initial FeO prediction model:

In [4]:
wholerock_file = "./data/wholerock.csv"
wholerock = mp.read_melt(wholerock_file, index_col=["name"])

x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

Next, we need to set up the object that handles the random sampling of errors in the Monte Carlo simulation. This is done with the *PEC_MC_parameters* class and it includes the following parameters for error propagation:

- **melt_errors**
        
    propagate errors on melt composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **olivine_errors**

    propagate errors on olivine composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **FeOi_errors**

    propagate errors on estimate melt initial FeO contents. Fixed errors can be provided either for the whole dataset, or per inclusion. Alternatively, an [FeOi_prediction object](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html) can be provided to propagate errors on predictions models.

- **Fe3Fe2**

    Propagate errors on modelled melt Fe<sup>2+</sup>/Fe<sup>3+</sup> ratios. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

- **Kd**

    Propagate errors on modelled olivine-melt Fe-Mg partition coefficients. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

By default errors are not propagated - you explicitely need to tell MagmaPEC to do so when initialising the *PEC_MC_parameters* object

In this example we will use all error propagation options, which means we need to provide melt and olivine composition errors. We import these from .csv files containing error data for individual inclusions and olivines. This is just an example with randomly generated errors, normally you should use analytical errors.

In [5]:
melt_errors_file = "./data/melt_errors.csv"
olivine_errors_file = "./data/olivine_errors.csv"

melt_errors = pd.read_csv(melt_errors_file, index_col=[0])
olivine_errors = pd.read_csv(olivine_errors_file, index_col=[0])

Make very sure that the elements in the error data have identical sorting to the melt and olivine dataframes, otherwise errors will be applied to the wrong elements. 
We can force this by sorting the columns of the error dataframes (or series) with the *elements* attributes of the melt and olivine MagmaFrames:

In [6]:
melt_errors = melt_errors[melt.elements]
olivine_errors = olivine_errors[olivine.elements]

Here's what they look like:

In [7]:
melt_errors.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
PI032-04-01,1.02,0.64,0.14,0.47,0.69,0.10,0.08,0.05,0.18,0.05,0.05,0.17,0.21,0.22,0.10,0.12
PI032-04-02,1.06,0.84,0.29,0.46,0.54,0.05,0.03,0.01,0.06,0.18,0.04,0.24,0.12,0.13,0.12,0.04
PI041-02-02,1.04,0.90,0.09,0.55,0.49,0.22,0.21,0.01,0.16,0.04,0.00,0.15,0.23,0.04,0.17,0.06
PI041-03-01,0.98,0.64,0.10,0.40,0.53,0.12,0.12,0.08,0.01,0.03,0.00,0.05,0.36,0.06,0.14,0.04
PI041-03-03,1.02,0.54,0.31,0.42,0.68,0.18,0.03,0.02,0.11,0.06,0.10,0.18,0.08,0.13,0.06,0.15


In [8]:
olivine_errors.head()

,SiO2,FeO,MgO,NiO,MnO,Al2O3,CaO
PI032-04-01,1.06,0.60,1.22,0.01,0.09,0.08,0.02
PI032-04-02,0.85,0.57,1.28,0.14,0.07,0.24,0.06
PI041-02-02,1.11,0.44,1.30,0.08,0.10,0.01,0.19
PI041-03-01,0.99,0.50,1.31,0.14,0.16,0.15,0.01
PI041-03-03,0.95,0.64,1.21,0.13,0.11,0.10,0.15


Together with the *FeOi_prediction* object, we pass these as arguments to the *PEC_MC_parameters* object. We also set *Fe3Fe2* and *Kd* to *True* in order to propagate their model errors.

In [9]:
mc_parameters = mpc.PEC_MC_parameters(melt_errors=melt_errors, olivine_errors=olivine_errors, FeOi_errors=FeOi_predict, Fe3Fe2=True, Kd=True, temperature=True)

Now we can create the Monte Carlo model with the *PEC_MC* object

In [10]:
pec_mc_model = mpc.PEC_MC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeOi_predict, MC_parameters=mc_parameters)

And finally run the Monte Carlo simulation with *n* iterations

In [11]:
pec_mc_model.run(n=40)

Monte Carlo loop
040/040
Equilibrating ... |██████████████████████████████| 100% [10/10] in 4.2s 
Correcting    ... |██████████████████████████████| 100% [10/10] in 12.9s 


Results are stored internally in the following attributes:

- **pec**: pandas DataFrame

    Average PEC extents (%) of the MC model and their one standard deviation errors.

- **inclusions_corr**: MagmaPandas Melt frame

    Averages of corrected melt inclusion compositions (wt. %)

- **inclusions_stddev**: pandas DataFrame

    One standard deviation errors on inclusions_corr (wt. %)

- **pec_MC**: pandas DataFrame

    PEC extents for individual iterations

- **inclusions_MC**: dictionary of MagmaPandas Melt frames

    corrected melt inclusion compositions for individual iterations.

In [12]:
pec = pec_mc_model.pec
inclusions_corrected = pec_mc_model.inclusions_corr
inclusions_errors = pec_mc_model.inclusions_stddev

pec_mc = pec_mc_model.pec_MC
inclusions_MC = pec_mc_model.inclusions_MC

In [13]:
pec

,pec,stddev
name,,
PI032-04-01,8.429927,4.400038
PI032-04-02,9.429042,4.773970
PI041-02-02,-0.804338,2.839375
PI041-03-01,11.960669,4.911413
PI041-03-03,10.632113,5.027331
PI041-05-04,-4.919785,2.861665
PI041-05-06,1.601953,3.545407
PI041-07-01,11.220096,4.844009
PI041-07-02,10.209749,4.391335


In [14]:
inclusions_corrected

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
name,,,,,,,,,,,,,,,,
PI032-04-01,48.974665,14.216788,7.196282,9.905579,10.202266,3.624333,0.662520,0.144112,2.482462,0.280820,0.000000,0.582205,1.396228,0.111677,0.155710,0.064353
PI032-04-02,48.363809,14.886235,7.026828,9.700264,10.268039,3.492956,0.925823,0.136860,2.596414,0.382665,0.000000,0.643052,1.257333,0.087223,0.191847,0.040651
PI041-02-02,49.131492,17.189230,4.557404,9.245858,10.143751,3.837282,1.019967,0.152095,2.805114,0.571482,0.000000,0.460185,0.687148,0.039611,0.123317,0.036063
PI041-03-01,46.088633,15.804956,6.879916,10.899040,10.678160,3.343396,1.156954,0.109385,3.135612,0.547532,0.000000,0.789147,0.329367,0.089029,0.094778,0.054094
PI041-03-03,45.450810,16.128484,6.598134,11.310218,10.693169,3.436284,1.183799,0.086589,3.241342,0.527906,0.000000,0.800150,0.322737,0.100971,0.067093,0.052314
PI041-05-04,47.960734,18.895538,3.405925,9.501497,9.247093,4.652959,1.640546,0.142263,2.514607,0.833688,0.000000,0.480654,0.460711,0.086625,0.120171,0.056988
PI041-05-06,46.343832,16.944093,4.452132,8.997968,11.564994,4.148770,1.457989,0.153863,3.668803,0.632094,0.000000,0.670164,0.641132,0.124801,0.133505,0.065859
PI041-07-01,45.596512,15.572121,6.883172,9.712319,11.582578,3.188643,1.289784,0.130081,3.565148,0.560008,0.000000,0.471958,1.120397,0.076854,0.162528,0.087898
PI041-07-02,46.128650,15.572221,6.550045,10.093937,11.531546,3.219264,1.348534,0.118764,3.564718,0.621947,0.000000,0.354760,0.630456,0.074243,0.135053,0.055862


In [15]:
inclusions_errors

,SiO2_stddev,Al2O3_stddev,MgO_stddev,CaO_stddev,FeO_stddev,Na2O_stddev,K2O_stddev,MnO_stddev,TiO2_stddev,P2O5_stddev,Cr2O3_stddev,CO2_stddev,H2O_stddev,F_stddev,S_stddev,Cl_stddev
name,,,,,,,,,,,,,,,,
PI032-04-01,0.700672,0.788486,1.357145,0.625139,0.435496,0.164936,0.098278,0.043240,0.171569,0.042367,0.000000,0.138803,0.172027,0.153330,0.093356,0.072502
PI032-04-02,0.635984,1.065275,1.424407,0.462308,0.298112,0.147213,0.045423,0.012185,0.124407,0.122128,0.000000,0.234705,0.088480,0.093863,0.111122,0.035407
PI041-02-02,0.793829,0.938624,0.979591,0.501948,0.410350,0.239260,0.184181,0.013321,0.197988,0.044487,0.000000,0.155905,0.233562,0.032321,0.138541,0.048244
PI041-03-01,0.567960,0.859470,1.557239,0.567303,0.166339,0.175953,0.129690,0.085233,0.116408,0.038182,0.000000,0.046932,0.291600,0.049837,0.111448,0.034696
PI041-03-03,0.557517,0.774783,1.569120,0.641989,0.228583,0.209358,0.054073,0.027925,0.152729,0.055564,0.000000,0.131856,0.055586,0.081465,0.047914,0.062572
PI041-05-04,0.678302,0.887166,0.849553,0.449395,0.310902,0.193892,0.142849,0.140305,0.080242,0.034042,0.000000,0.184296,0.158081,0.099842,0.011305,0.012454
PI041-05-06,0.801583,1.075820,1.014791,0.412772,0.388681,0.389032,0.221616,0.187878,0.167373,0.059307,0.000000,0.204210,0.281854,0.130613,0.059468,0.024965
PI041-07-01,0.464395,0.939907,1.417700,0.512431,0.273804,0.128596,0.057152,0.053194,0.135439,0.026433,0.000000,0.048315,0.237396,0.093153,0.031307,0.098482
PI041-07-02,0.661935,0.714356,1.366864,0.434750,0.237183,0.138173,0.173095,0.058581,0.138372,0.024410,0.000000,0.178120,0.210618,0.002789,0.042041,0.068106


In [20]:
pec_mc.head()

name,PI032-04-01,PI032-04-02,PI041-02-02,PI041-03-01,PI041-03-03,PI041-05-04,PI041-05-06,PI041-07-01,PI041-07-02,PI052-01-02
iteration,,,,,,,,,,
0,12.086981,13.642188,-1.2,15.829224,16.036627,-5.39978,1.010843,14.200342,9.760767,-4.479761
1,19.202637,19.360352,3.944214,24.080566,22.949438,1.342261,8.358716,22.918506,20.429785,-3.721387
2,3.000647,5.016333,-4.525146,7.623315,5.493384,-8.620508,-3.07489,6.818149,6.082495,-12.643933
3,11.028467,9.754126,1.48252,13.859741,15.212988,-1.23584,4.285791,16.220825,15.084375,-7.667993
4,6.401306,3.128052,-3.460303,6.761182,3.777856,-4.190527,0.392468,5.482495,7.154871,-11.367969


The dataframes in *inclusions_MC* also have *isothermal_equilibration* and *Kd_equilibration* columns. These columns show if equilibration during stage [1] and [2] respectively was successful. In extreme cases, error propagation of melt and olivine compositions by random sampling can result in melt-olivine pairs that cannot be equilibrated without crystallising more than the entire inclusion or exchanging more Mg or Fe than is present in the inclusion. If that is the case, no corrected compositions are calculated and eiher the isothermal- of Kd-equilibration column is set to *False*.

In [17]:
inclusions_MC["PI032-04-01"].head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,isothermal_equilibration,Kd_equilibration
iteration,,,,,,,,,,,,,,,,,,
0,49.400296,13.226125,8.035545,9.782201,10.409923,3.523617,0.585936,0.178366,2.378305,0.334652,0.0,0.434459,1.441352,0.0,0.269223,0.0,NaN,NaN
1,47.703928,13.116121,10.551913,9.078296,10.668488,3.471678,0.567647,0.109088,2.484392,0.215764,0.0,0.625669,1.236557,0.046849,0.12361,0.0,NaN,NaN
2,49.034346,15.51288,5.473598,10.723741,9.562159,3.78794,0.832688,0.210658,2.384656,0.345957,0.0,0.730455,1.260252,0.102069,0.038599,0.0,NaN,NaN
3,48.802271,13.563949,8.625841,9.578546,10.166568,3.324619,0.622145,0.158414,2.447163,0.345179,0.0,0.522051,1.344214,0.467414,0.031625,0.0,NaN,NaN
4,50.024637,13.940714,6.362948,9.622359,10.424235,3.771884,0.563701,0.114497,2.529173,0.277057,0.0,0.456753,1.419312,0.128764,0.316908,0.047059,NaN,NaN
